<a href="https://colab.research.google.com/github/serereuk-tf/Tensorflow-Example/blob/main/Day1/TF_Data%26Augmentation_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이전에 사용했던 예제를 다시 올려두고 시작합시다

In [1]:
#unzip the Files
!unzip -qq /content/train_tfrecord.zip

In [10]:
import glob
import numpy as np
from PIL import Image
import tensorflow as tf

# Tensorflow에서 알아서 최적화를 해주는 좋은 기능 #
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [13]:
def _parse_function(serialized_item):
    """
    TF Records에 있는 내용물들을 압축해제 하는 방법
    이미지가 String으로 나오기 때문에 추가적으로 더 작업을 해줘야한다.
    tf.io.decode_png를 통해서 Stinrg -> Image Array
    Output:
        * Image : Tensor
        * Label : Tensor
    """
    parsed_ = tf.io.parse_example(
        serialized=serialized_item,
        features={
            "image": tf.io.FixedLenFeature([], dtype=tf.string),
            "label": tf.io.FixedLenFeature([], dtype=tf.int64)
        }
    )
    image = parsed_['image']
    image = tf.io.decode_png(image, channels=3)
    label = parsed_['label']
    return image, label

# Image Augmentation을 적용하는 방법은 크게 2 가지

1. tf dataset에서 tf.image를 사용하여 변경하는 방법
2. tf.keras.layers를 활용하는 방법

In [14]:
# 1. Dataset을 활용한 방법

def Augmentation(image):
    """
    랜덤 Flip_LR를 적용한 모습 
    Output:
        * Image : Flipped Image
    """
    image = tf.image.random_flip_left_right(image)
    return image

# 2. tf.keras.layer를 활용하는 방법
Augmentation2 = tf.keras.Sequential(
    [tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal')])


In [12]:
# TF Record 읽기
train_dataset = tf.data.TFRecordDataset(
    glob.glob('tfrecord_cifar100/train/tfr_*'))
# Parsing
train_dataset = train_dataset.map(_parse_function, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(Augmentation, num_parallel_calls=AUTOTUNE)